In [105]:
import autoencoder
import utils

import pandas as pd
import torch

from sklearn.model_selection import KFold

## Prepare data:
#### Read joined data (pre + post treatment)

In [4]:
lincs_joined_df = pd.read_parquet("data/lincs_pretreatment.parquet")
kaggle_joined_df = pd.read_parquet("data/kaggle_pretreatment.parquet")
test_joined_df = pd.read_parquet("data/test_pretreatment.parquet")
print(f"lincs_joined_df = {lincs_joined_df.shape}\nkaggle_joined_df = {kaggle_joined_df.shape}\ntest_joined_df = {test_joined_df.shape}")

lincs_joined_df = (107404, 1842)
kaggle_joined_df = (602, 1841)
test_joined_df = (255, 921)


#### Kaggle provided data

In [8]:
de_train = pd.read_parquet('data/de_train.parquet')
id_map = pd.read_csv('data/id_map.csv',index_col='id')
print(f"de_train = {de_train.shape}\nid_map = {id_map.shape}")

de_train = (614, 18216)
id_map = (255, 2)


#### Define features of interest and sort data accordingly.

In [9]:
features = ['cell_type', 'sm_name']
multiindex_features = [("label",'cell_type'),("label",'sm_name')]

transcriptome_cols = de_train.columns[5:]
landmark_cols = kaggle_joined_df["post_treatment"].columns
print(f"transcriptome_cols = {transcriptome_cols.shape}\nlandmark_cols = {landmark_cols.shape}")

transcriptome_cols = (18211,)
landmark_cols = (918,)


In [133]:
unique_sm_name = pd.concat([lincs_joined_df[("label","sm_name")],kaggle_joined_df[("label","sm_name")]]).drop_duplicates().reset_index(drop=True)
unique_cell_type = pd.concat([lincs_joined_df[("label","cell_type")],kaggle_joined_df[("label","cell_type")]]).drop_duplicates().reset_index(drop=True)
print(f"Number of unique molecules = {len(unique_sm_name)}.\nNumber of unique cell types = {len(unique_cell_type)}.")

Number of unique molecules = 1896.
Number of unique cell types = 36.


In [10]:
# We only need to sort these two dataframes because they represent the same underlying dataset.
de_train = de_train.query("~control").sort_values(features)
kaggle_joined_df = kaggle_joined_df.sort_values(multiindex_features)
# Sanity check that these dfs align.
genes_align = (kaggle_joined_df["post_treatment"] == de_train[landmark_cols]).all(axis=None)
labels_align = (kaggle_joined_df["label"][features] == de_train[features]).all(axis=None)
genes_align and labels_align

True

#### CV splits

In [80]:
eval_cells_only_df = kaggle_joined_df[kaggle_joined_df["label"]["cell_type"].isin(["B cells", "Myeloid cells"])][multiindex_features]
len(eval_cells_only_df)

30

In [84]:
fold_to_eval_df = {}
skf = KFold(n_splits=3, random_state=42, shuffle=True)
for i,(t,v) in enumerate(skf.split(eval_cells_only_df)):
    fold_to_eval_df[i] = eval_cells_only_df.iloc[v]

for i, df in fold_to_eval_df.items():
    print(f"fold = {i} of shape {df.shape}")

fold = 0 of shape (10, 2)
fold = 1 of shape (10, 2)
fold = 2 of shape (10, 2)


In [87]:
def make_mask(fold):
    val = fold_to_eval_df[fold]
    return kaggle_joined_df[("label","sm_name")].isin(val[("label","sm_name")]) & kaggle_joined_df[("label","cell_type")].isin(val[("label","cell_type")])

print("Using fold 0 as validation set:")
print(f"Train data = {pd.concat([kaggle_joined_df[~make_mask(0)],lincs_joined_df]).shape}")
print(f"Validation data = {kaggle_joined_df[make_mask(0)].shape}")

Using fold 0 as validation set:
Train data = (107994, 1843)
Validation data = (12, 1841)


In [172]:
cell_type_map = {v: k for k,v in unique_cell_type.to_dict().items()}
sm_name_map = {v: k for k,v in unique_sm_name.to_dict().items()}
def joined_df_to_tensor(df):
    ct = df[("label","cell_type")].map(cell_type_map)
    sm = df[("label","sm_name")].map(sm_name_map)
    return {"cell_type":torch.tensor(ct.to_numpy()),
            "sm_name":torch.tensor(sm.to_numpy()),
            "pre_treatment":torch.tensor(df["pre_treatment"].to_numpy(),dtype=torch.float),
            "post_treatment":torch.tensor(df["post_treatment"].to_numpy(),dtype=torch.float)}

In [173]:
class Translator(torch.nn.Module):
    def __init__(self,config):
        super(Translator,self).__init__()
        # This will eventually be changed to a GNN
        self.sm_embed = torch.nn.Embedding(num_sm_name, config["sm_emb_size"])

        # This needs to be able to handle out of dictionary
        self.cell_embed = torch.nn.Embedding(num_cell_type, config["cell_emb_size"])

        self.config = config
        input_dim = config["sm_emb_size"] + config["cell_emb_size"] + config["latent_dim"]
        self.translation = utils.make_sequential(input_dim,config["hidden_dim"],config["latent_dim"],config["dropout"])

    def forward(self,inp,z):
        pemb = self.sm_embed(inp["sm_name"])
        cemb = self.cell_embed(inp["cell_type"])
        x = torch.cat((pemb, cemb, z), dim=1)
        return self.translation(x)
        
class RNVAE(torch.nn.Module):
    def __init__(self,config):
        super(RNVAE,self).__init__()
        self.vae = autoencoder.AutoEncoder(target_dim=len(landmark_cols),config=config)
        self.translator = Translator(config)

    def forward(self,inp):
        assert inp["pre_treatment"].shape == inp["post_treatment"].shape
        latent = self.vae.latent(inp["pre_treatment"])
        z_prime = self.translator(inp,latent["z"])
        x_hat = self.vae.decode(z_prime)
        return {"x_hat":x_hat, "mu": latent["mu"], "log_var":latent["log_var"]}

    def loss_function(self,fwd,inp):
        return self.vae.loss_function(fwd,inp["post_treatment"])

example_config = {
    "sm_emb_size": 64,
    "cell_emb_size": 32,
    "latent_dim": 256,
    "hidden_dim": 512,
    "dropout": .1,
    "kld_weight": 1,
}
inp = joined_df_to_tensor(kaggle_joined_df[make_mask(0)])
model = RNVAE(example_config)
model(inp)

{'x_hat': tensor([[-0.1681, -0.0463,  0.0038,  ...,  0.0029,  0.0087, -0.0397],
         [ 0.0067,  0.0804, -0.0268,  ..., -0.0145, -0.0012, -0.0693],
         [-0.1500,  0.0442, -0.0124,  ...,  0.0250, -0.1088, -0.0850],
         ...,
         [-0.1523,  0.0916, -0.0342,  ...,  0.1589, -0.0715, -0.1221],
         [-0.0771,  0.0482, -0.0566,  ...,  0.0982, -0.0791, -0.0439],
         [-0.0043, -0.0217, -0.0426,  ..., -0.0292, -0.0679, -0.1082]],
        grad_fn=<AddmmBackward0>),
 'mu': tensor([[-0.2404, -1.0101,  0.8763,  ...,  0.1315, -0.7874, -0.2741],
         [ 0.1063, -0.6510,  1.3623,  ...,  0.1076, -0.5184, -0.4736],
         [ 0.0214, -0.4545,  1.3380,  ..., -0.0906, -0.6907, -0.4632],
         ...,
         [-1.0559,  0.0772,  1.2063,  ..., -0.5359,  0.0815, -0.2967],
         [-0.8867, -0.5100,  0.5140,  ..., -0.2064, -0.0143, -0.4027],
         [-0.9105, -0.4386,  0.7304,  ..., -0.2329,  0.4902, -0.4755]],
        grad_fn=<AddmmBackward0>),
 'log_var': tensor([[-7.6624e-01,